In [ ]:
# to automatically reload modules who's content has changed
%load_ext autoreload
%autoreload 2
# configure matplotlib
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

In [ ]:
from task_utils import *

In [ ]:
import funbo as fb
import funbo.plotting as fp

In [ ]:
import simulated_annealing as sa

# A function fitting example

In [ ]:
domain_bounds = [(0, 6), (0, 7)]
range_bounds = (-1, 2.5)
def to_fit(X):
    ''' from https://github.com/fmfn/BayesianOptimization/issues/18 '''
    x, y = X[:,0], X[:,1]
    a = np.exp(-( (x - 2)**2/0.7 + (y - 4)**2/1.2) + (x - 2)*(y - 4)/1.6 )
    b = np.exp(-( (x - 4)**2/3 + (y - 2)**2/2.) )
    c = np.exp(-( (x - 4)**2/0.5 + (y - 4)**2/0.5) + (x - 4)*(y - 4)/0.5 )
    d = np.sin(3.1415 * x)
    e = np.exp(-( (x - 5.5)**2/0.5 + (y - 5.5)**2/.5) )
    val = 2*a + b - c + 0.17 * d + 2*e
    #return val.reshape(-1, 1)
    return val

In [ ]:
def plot_optimisation(net, state_record, num_frames, show_to_fit=False, frame_duration=200, display=True, interactive=False):
    g = fb.RegularGrid(50, domain_bounds, traverse_order='big')
    X, Y = g.meshgrid(cartesian_index=False)
    Z = g.fun_on_grid(to_fit)
    goal_XYZ = (X, Y, Z)
    if not show_to_fit:
        goal_XYZ = None
    
    num_frames = num_frames
    amplitudes = [a for i, a in enumerate(state_record) if i % (len(state_record)//num_frames) == 0]
    args = dict(
        net=net,
        amplitudes=amplitudes,
        goal_XYZ=goal_XYZ,
        axes_names=('x1', 'x2', 'y'),
        axes_limits=(*domain_bounds, range_bounds),
    )
    if interactive:
        sa.plot_elastic_net_animation_interactive(**args)
    else:
        return sa.plot_elastic_net_animation(**args, frame_duration=frame_duration, display=display)

In [ ]:
def optimise_simulated_annealing(h):
    np.random.seed(0)
    grid = fb.RegularGrid(num_values=h.num_CP, bounds=domain_bounds)
    net = fb.ElasticNet(grid, elastic_stiffness=h.elastic_stiffness, range_bounds=range_bounds)
    fit_cost = lambda net, amplitudes: np.zeros(shape=net.grid.shape)
    def perturbation(step_size):
        return np.random.uniform(-step_size, step_size, size=grid.shape)
    candidate_func = lambda current_amplitudes: np.clip(current_amplitudes + perturbation(h.initial_step), *net.range_bounds)
    cooling_schedule = lambda i: sa.temperature_exponential_decay(i, M=h.M, factor=h.factor, T_0=h.T_0, T_min=h.T_min)
    best_a, best_E, state_record, acceptance_record = sa.elastic_net_simulated_annealing(net, fit_cost, initial_amplitudes=net.random_amplitudes(),
                                                                                         max_its=h.max_its, CP_batch_size=h.CP_batch_size,
                                                                                         candidate_func=candidate_func, cooling_schedule=cooling_schedule)
    print('acceptance rate = {}'.format(np.count_nonzero(acceptance_record)/acceptance_record.size))
    print('best E = {}'.format(best_E))
    return net, best_a, state_record

In [ ]:
class hyper_params:
    max_its=100
    
    initial_step=0.3 # not adjustable yet
    #step_alter_chunk=10
    
    M=1 # lower seems to be more reliably good
    factor=0.8
    T_0=1
    T_min=0.2

    CP_batch_size=1
    
    num_CP=10
    elastic_stiffness=1

net, best_a, state_record = optimise_simulated_annealing(hyper_params)

In [ ]:
#plot_optimisation(net, [a for a, e in state_record], num_frames=30)
ani = plot_optimisation(net, [a for a, e in state_record], num_frames=20, display=False)
sa.display_video(ani)
plt.close()
#sa.save_video(ani, 'simulated_annealing_elastic.mp4')

In [ ]:
def _():
    g = fb.RegularGrid(50, domain_bounds, traverse_order='big')
    X, Y = g.meshgrid(cartesian_index=False)
    Z = g.fun_on_grid(to_fit)
    goal_XYZ = (X, Y, Z)
    goal_XYZ = None
    
    sa.plot_elastic_net(net, amplitudes=best_a, goal_XYZ=goal_XYZ, axes_names=('x1', 'x2', 'y'), axes_limits=(*domain_bounds, range_bounds))
_()

In [ ]:
def _():
    fig, ax = plt.subplots(figsize=(12, 8))
    xs = np.arange(len(state_record))
    ax.plot(xs, [e for _, e in state_record])
    ax.set_ylabel('E')
    ax.set_xlabel('iteration')
_()

In [ ]:
def _():
    g = fb.RegularGrid(50, domain_bounds, traverse_order='big')
    X, Y = g.meshgrid(cartesian_index=False)
    Z = g.fun_on_grid(to_fit)
    goal_XYZ = (X, Y, Z)
    #goal_XYZ = None
    
    
    #sa.plot_elastic_net(g, net, amplitude=net.random_amplitudes(), goal_z=Z, axes_names=('x1', 'x2', 'y'))
    
    """
    amplitudes = []
    for i in range(4):
        amplitudes.append(net.random_amplitudes())
    """
    
    num_frames = 20
    amplitudes = [a for i, (a, e) in enumerate(state_record) if i % (len(state_record)//num_frames) == 0]
    sa.plot_elastic_net_animation_interactive(net, amplitudes=amplitudes, goal_XYZ=goal_XYZ, axes_names=('x1', 'x2', 'y'), axes_limits=(*domain_bounds, range_bounds))
_()

# Gradient Descent

Advantages over Simulated Annealing:
- less hyperparameters
- less sensitive to hyperparameter settings (adagrad)
- more accurate result
- faster to converge to a decent solution

Disadvantages over simulated annealing:
- local search rather than global search
- requires gradient calculation which is more expensive than value calculation for GPs

In [ ]:
def optimise_GD(h):
    np.random.seed(0)
    grid = fb.RegularGrid(num_values=h.num_CP, bounds=domain_bounds)
    net = fb.ElasticNet(grid, elastic_stiffness=h.elastic_stiffness, range_bounds=range_bounds)
    if h.fit_to_func:
        Z = grid.fun_on_grid(to_fit)
        fit_cost_gradient = lambda amplitudes: amplitudes - Z # -(Z - amplitudes)
    else:
        fit_cost_gradient = lambda amplitudes: np.zeros(shape=net.grid.shape)
    cost_gradient = lambda state: fit_cost_gradient(state) + net.elastic_potentials_gradient(state)
    state, state_record = fb.gradient_descent(cost_gradient, initial_state=net.random_amplitudes(), max_its=h.max_its, step_size=h.step_size, adaptive=h.adaptive, record_state=True)
    return net, state, state_record

In [ ]:
class hyper_params:
    max_its = 100
    adaptive = False
    step_size = lambda i: 0.1
    
    num_CP=20
    elastic_stiffness=1
    fit_to_func=False

net, state, state_record = optimise_GD(hyper_params)

In [ ]:
#plot_optimisation(net, state_record, show_to_fit=True, num_frames=10, interactive=True)
ani = plot_optimisation(net, state_record, num_frames=20, display=False)
sa.display_video(ani)
plt.close()
#sa.save_video(ani, 'gradient_descent_elastic.mp4')

In [ ]:
class hyper_params:
    max_its = 100
    adaptive = True
    step_size = lambda i: 1
    num_CP=20
    elastic_stiffness=1
    fit_to_func=False

net, state, state_record = optimise_GD(hyper_params)

In [ ]:
#plot_optimisation(net, state_record, num_frames=30)
ani = plot_optimisation(net, state_record, num_frames=20, display=False)
sa.display_video(ani)
plt.close()
#sa.save_video(ani, 'adagrad_elastic.mp4')

In [ ]:
plot_optimisation(net, state_record, num_frames=30)

In [ ]:
class hyper_params:
    max_its = 20
    adaptive = True
    step_size = lambda i: 1
    
    num_CP=20
    elastic_stiffness=0.01
    fit_to_func=True

net, state, state_record = optimise_GD(hyper_params)

In [ ]:
plot_optimisation(net, state_record, show_to_fit=True, num_frames=20, interactive=True)